In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

AUTOTUNE = tf.data.AUTOTUNE

from shared_funcs import multi_label_binary_encode_tensor, multi_label_binary_decode_tensor, get_waveform, split_into_windows, split_into_sequences 

2024-12-04 21:32:00.322253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 21:32:00.336350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 21:32:00.340654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 21:32:00.352695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 21:32:01.740475: W tensorflow/compiler/tf2

In [2]:
ALL_LABELS = tf.constant(['Rctrl', 'p', 'esc', 'g', 'slash', 'down', '7', 'equal', 'w', 'a', 'dash', 'caps', 'l', 'd', 'backspace', 'bracketclose', 'z', '1', 'end', 'Rshift', 'comma', 'c', 'tab', 'b', 'j', 'right', 'Lctrl', 'n', 't', 'f', 'm', 'o', 'apostrophe', 'y', '8', 'space', 'backslash', 's', '9', 'i', 'r', 'bracketopen', 'semicolon', 'q', '5', 'k', '3', 'x', '4', '6', '2', 'Lshift', 'left', 'backtick', 'enter', 'fullstop', 'e', '0', 'h', 'v', 'up', 'u', 'delete'], dtype=tf.string)


I0000 00:00:1733373124.086681 1757895 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733373124.088166 1757895 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733373124.089622 1757895 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733373124.241876 1757895 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

### Load Datasets

In [3]:
# Load MKA Dataset
# mka_path = os.path.dirname(os.getcwd()) + "/data-manipulation/mka_dataset"
# mka_dataset = tf.data.Dataset.load(mka_path)

# Load supplementary dataset
supp_path = os.path.dirname(os.getcwd()) + "/data-manipulation/supp_dataset"
supp_dataset = tf.data.Dataset.load(supp_path)

supp_dataset = supp_dataset.shuffle(64)

# mka_dataset.cache()
supp_dataset.cache()
# mka_dataset = mka_dataset.prefetch(tf.data.AUTOTUNE)
supp_dataset = supp_dataset.prefetch(AUTOTUNE)

# print(f"MKA Dataset {mka_dataset.element_spec}\nLength: {tf.data.experimental.cardinality(mka_dataset).numpy()}")
print(f"Supp Dataset {supp_dataset.element_spec}\nLength: {tf.data.experimental.cardinality(supp_dataset).numpy()}")

spectrogram_dataset = supp_dataset

# Add channel dimension for conv network
spectrogram_dataset = spectrogram_dataset.map(
    lambda spectrogram, label: (tf.expand_dims(spectrogram, axis=-1), label),
    num_parallel_calls=tf.data.AUTOTUNE
)

Supp Dataset (TensorSpec(shape=(39, 1103), dtype=tf.float32, name=None), TensorSpec(shape=(39, 63), dtype=tf.int32, name=None))
Length: 752


### Prepare Dataset for Training

In [4]:
dataset_size = sum(1 for _ in spectrogram_dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)

# Split into train val and test datasets
train_dataset = spectrogram_dataset.take(train_size)
remaining_dataset = spectrogram_dataset.skip(train_size)
val_dataset = remaining_dataset.take(val_size)
test_dataset = remaining_dataset.skip(val_size)

2024-12-04 21:32:04.484917: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Calculate Class Weights
Since the dataset may be imbalanced with different classes, especially with the no key pressed class, we must calculate the class weights so the model is not biased towards the majority class

In [5]:
def calculate_class_weights(labels):
    print(labels.shape)
    num_samples, num_classes = labels.shape
    class_counts = np.sum(labels, axis=0)
    total_counts = np.sum(class_counts)
    
    class_weights = {i: total_counts / (num_classes * class_counts[i]) for i in range(num_classes)}
    return class_weights

# Example usage
all_labels = np.concatenate([label.numpy() for _, label in train_dataset])  # Gather all labels
class_weights = calculate_class_weights(all_labels)
print("Class Weights:", class_weights)

(23439, 63)
Class Weights: {0: 6.375283446712018, 1: 1.4793475401210208, 2: 3.0777230432402845, 3: 0.970151828847481, 4: 3.0427489177489178, 5: inf, 6: 13.388095238095238, 7: 8.925396825396826, 8: 0.9666494756747465, 9: 0.2715637979329663, 10: 2.1593701996927805, 11: 3.150140056022409, 12: 0.5897839311936228, 13: 0.5625250100040016, 14: 0.7102437792092965, 15: 4.781462585034014, 16: 2.2691686844229215, 17: 4.389539422326307, 18: inf, 19: 3.4328449328449326, 20: 3.0427489177489178, 21: 0.6848130556570454, 22: 44.62698412698413, 23: 1.3125583566760037, 24: 11.156746031746032, 25: inf, 26: 5.820910973084886, 27: 0.37292744395808464, 28: 0.2523674879942552, 29: 1.3455372098588179, 30: 0.7783776301218162, 31: 0.409421872724625, 32: 3.523182957393484, 33: 0.7992892679459843, 34: 7.046365914786968, 35: 0.11774929848808477, 36: 3.4328449328449326, 37: 0.302898082309847, 38: 6.375283446712018, 39: 0.32144286285942947, 40: 0.39786315714993276, 41: 11.641821946169772, 42: 3.0427489177489178, 43: 

2024-12-04 21:32:04.629856: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/tmp/ipykernel_1757895/1588371172.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  class_weights = {i: total_counts / (num_classes * class_counts[i]) for i in range(num_classes)}


### Batch and Prefetch datasets

In [6]:
# Batch and prefetch
train_dataset = train_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)

# Model

In [7]:
print(spectrogram_dataset.element_spec)
model = keras.Sequential([
    layers.Input(shape=(39, 1103, 1)),
    layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(1, 2)),
    layers.Dropout(0.3),

    layers.Reshape((39, -1)),
    layers.TimeDistributed(layers.Dense(128, activation='relu')),
    layers.TimeDistributed(layers.Dense(63, activation="sigmoid"))
])

model.summary()

(TensorSpec(shape=(39, 1103, 1), dtype=tf.float32, name=None), TensorSpec(shape=(39, 63), dtype=tf.int32, name=None))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 39, 1103, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 39, 1103, 128)  │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 39, 551, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 39, 551, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 39, 70528)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 39, 128)        │     9,027,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 39, 63)         │         8,127 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,073,151 (34.61 MB)

 Trainable params: 9,073,151 (34.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy'] # , keras.metrics.Precision()
)

model.fit(
    train_dataset,
    epochs=30,  # Adjust as needed
    validation_data=val_dataset,
    class_weight=class_weights
)

Epoch 1/30


I0000 00:00:1733373127.631863 1758043 service.cc:146] XLA service 0x7fe17c009c60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733373127.631886 1758043 service.cc:154]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-12-04 21:32:07.744838: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-04 21:32:08.763946: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 1/19 ━━━━━━━━━━━━━━━━━━━━ 6:15 21s/step - accuracy: 0.0024 - loss: 2.0516

I0000 00:00:1733373145.904971 1758043 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.0070 - loss: 1.4705 - val_accuracy: 0.0417 - val_loss: 0.1737
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.3656 - loss: 0.1740 - val_accuracy: 0.1856 - val_loss: 0.1896
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3689 - loss: 0.1731 - val_accuracy: 0.2947 - val_loss: 0.1699
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3772 - loss: 0.1705 - val_accuracy: 0.1874 - val_loss: 0.2063
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3775 - loss: 0.1718 - val_accuracy: 0.2434 - val_loss: 0.1896
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3807 - loss: 0.1694 - val_accuracy: 0.2496 - val_loss: 0.1845
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3780 - loss: 0.1751 - val_accuracy: 0.3159 - val_loss: 0.1625
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3808 - loss: 0.1731 - val_accuracy: 0.2421 - val_loss: 0.1

## Evaluate Model

In [9]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Loss: {loss}, Accuracy: {accuracy}")


# predictions = model.predict(new_data)

# threshold = 0.5
# binary_predictions = (predictions > threshold).astype(int)

# # Output predictions
# print("Raw Predictions:", predictions)
# print("Binary Predictions:", binary_predictions)

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.3163 - loss: 0.1658  
Loss: 0.1660735309123993, Accuracy: 0.31477731466293335


## Optimize Model